(LSE - ST456 DEEP LEARNING - WT2024)

# Logistic regression model

This notebook implements a **logistic regression model from scratch** applied for a **binary classification** problem (Titanic example). It is intended to be used for self-study and as a suplement to lecture content.

At the end, we show how to specify the same model using [TensorFlow Functional API](https://www.tensorflow.org/guide/keras/functional_api) (also known as *low-level API*)


---

## Step 0: software setup

In [ ]:
# importing necessary modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display
import tensorflow as tf

## Step 1: data loading and exploration

TensorFlow provides a version of the Titanic dataset; however, in this example we use a **modified version of the Titanic dataset** from [Kaggle](https://www.kaggle.com/azeembootwala/titanic). This version was adapted for logistic regression, and is available at the `data` folder. There are two files, namely:

* **`train_data.csv`**: a dataset of 792 instances and 16 features. The `survived` column is the target variable.

* **`testdata.csv`**: a dataset of 100 instances and 16 features. The `survived` column is the target variable.

Arrangements for both files:

* The `parch` and `sibsp` columns from the original data set were replaced by a single column called `Family size`.
* All categorical data, like `Embarked` and `pclass` have been re-encoded using **one-hot encoding**.
* Four more columns have been added, **re-engineered** from the `Name` column to `Title1` to `Title4` (Mr, Mrs, Master, Miss) signifying males and females depending on whether they were married or not. An additional analysis to see i) if `Married` people had more survival instincts or not, and ii) is the trend similar for both genders.
* All **missing values** have been filled with a median of the column values.
* All real valued data columns have been **normalised**.

In [ ]:
# loading the training dataset
# REMEMBER to upload this dataset into Colab
df1 = pd.read_csv('/content/titanic_train_data.csv')
# format (number of rows and columns)
df1.shape

In [ ]:
# basic information about the dataset (columns, data types, missing data)
df1.info()

The **target** variable  is `Survived`; all other columns are **features**.

A short description:

* `Sex`: 0 or 1 => male or female
* `Age`: value rescaled between 0 and 1
* `Fare`: ticket price rescaled between 0 and 1
* `Pclass_1` .. `Pclass_3`: Passenger class one-hot encoded
* `Family_size`: value rescaled between 0 and 1.
* `Title_1 .. Title_4`: mr, mrs, master, miss one-hot encoded
* `Emb_1 .. Emb_3`: Embark location one-hot encoded.

---

In [ ]:
# example instances
df1.sample(5)

---

## Step 2: data pre-processing

Let's first remove some unnecessary columns (not useful for the model) from the training dataset.

In [ ]:
# removing unnecessary columns
# Unnamed and PassengerID won't influence our model
df = df1.drop(['Unnamed: 0', 'PassengerId'], axis=1)

Let's **split the training dataset into features (`X`) and target (`Y`) variable**. We have a total of 792 examples. Therefore, the shape for `Y` is `(𝑚,1)` where `𝑚 = 792`. For `X` we expect `(𝑚, 14)`, where the columns are the features.

In [ ]:
# splitting the training dataset into features (X) and target (Y)
X_train = df.iloc[:,1:].to_numpy()
Y_train = df['Survived'].to_numpy()

In [ ]:
# Checking the structure
X_train.shape, Y_train.shape

In [ ]:
# transposing data - for compatibility
X_train_T = np.transpose(X_train)
Y_train_T = np.transpose(Y_train)
# Checking the structure
X_train_T.shape, Y_train_T.shape

Let's do the same with the **testing/validation dataset**.

In [ ]:
# loading the testing dataset
# REMEMBER to upload this dataset into Google Colab
df2 = pd.read_csv('/content/titanic_test_data.csv')
df2.shape

In [ ]:
# removing unecessary features
df2 = df2.drop(['Unnamed: 0', 'PassengerId'], axis=1)
# splitting the testing features (X) and labels (Y)
X_test = df2.iloc[:,1:].to_numpy()
Y_test = df2['Survived'].to_numpy()
# transposing data - for compatibility
X_test_T = np.transpose(X_test)
Y_test_T = np.transpose(Y_test)
# Checking the structure
X_test_T.shape, Y_test_T.shape

---

## Step 3: regression model (from scratch)

We will implement a logistic regression model from scratch, going through all the necessary steps (pipeline) for training and testing the model.

### 3.1: model defintion

Let's start by defining the **activation function**. We will use a custom `sigmoid` function for now.

In [ ]:
# custom sigmoid activation function
def sigmoid(Z):
    A = 1 / (1 + np.exp(-Z))
    return A

The next step is the **forward function**, which implements the dot product and makes use of the activation function.

We can split these in two steps:

$$Z = WX + b$$
$$A = \sigma(Z)$$


In [ ]:
# custom forward pass function
def forward(X, W, b):
    Z = np.dot(W.T, X) + b
    A = sigmoid(Z)
    return A

The **loss function** should be a **binary cross entropy**, as we have only two target classes (`survided = 1` or `0`).

$$loss = -\frac{1}{m}\sum_{i=1}^{m} y\log(A) + (1 - y)\log(1 - A)$$


In [ ]:
# custom loss function
# epsilon is a small value we add to avoid log(0) calculation
def loss(A, Y, epsilon = 1e-15):
    m = len(A)
    l = -1/m * np.sum(Y * np.log(A + epsilon) + (1 - Y) * np.log(1 - A + epsilon))
    return l

Next is the **backwards pass**. For this, we would need to differentiate the loss function with `W` and `b`.

$$\frac{\partial loss}{\partial W} \sum_{i=1}^{m} X(A - Y)\top$$

$$\frac{\partial loss}{\partial b} \sum_{i=1}^{m} (A - y)$$


In [ ]:
# custom backward pass function
def backward(X, Y, A):
    m = len(yhat)
    dW = 1/m * np.dot(X, (A - Y).T)
    db = 1/m * np.sum(A - Y)
    return (dW, db)

This step implements the **backpropagation function** for updating weights and bias.

In [ ]:
# weights and bias update
def update(W, b, dW, db, learning_rate = 0.01):
    W = W - learning_rate * dW
    b = b - learning_rate * db
    return (W, b)

As the activation function returns a probability between 0 and 1, we need a custom function to round values <= 0.5 to 0 and values > 0.5 to 1.

In [ ]:
# custom round function
def roundValue(A):
    return np.uint8(A > 0.5)

The last step is the definition of our **accuracy metric**.

In [ ]:
# custom accuracy function
def accuracy(yhat, Y):
    return round(np.sum(yhat==Y) / len(yhat) * 1000) / 10

### 3.2: model instantiation and training

In [ ]:
# initialising model parameters

# random seed (for reproducibility)
np.random.seed(2024)

# we have 14 features in the dataset
W = 0.01 * np.random.randn(14)
# and a constant bias
b = 0

# checking initial parameters
print("Initial weights:\n", W, "\n\nInitial bias: ", b)

In [ ]:
# training hyperparameters
num_iterations = 200 # number of training steps (epochs)
lr = 0.01            # learning rate

In [ ]:
# we will record loss and accuracy for plotting
losses, acces = [], []
losses_test, acces_test = [], []
# main training loop
for i in range(num_iterations):
    # forward pass
    A = forward(X_train_T, W, b)
    # loss calculation
    l = loss(Y_train_T, A)
    # round the predicted value
    yhat = roundValue(A)
    # accuracy calculation
    acc = accuracy(yhat, Y_train_T)
    # backpropagation pass - update weights and bias
    dW, db = backward(X_train_T, Y_train_T, A)
    W, b = update(W, b, dW, db, learning_rate=lr)
    # keep record of loss and accurcy
    losses.append(l)
    acces.append(acc)

    # Compute it on the test set
    A_test = forward(X_test_T, W, b)
    l_test = loss(Y_test_T, A_test)
    yhat_test = roundValue(A_test)
    acc_test = accuracy(yhat_test, Y_test_T)
    losses_test.append(l_test)
    acces_test.append(acc_test)
    # checkpoint every 50 iterations
    if i % 20 == 0:
        print('Loss on train:', l, f'\tAccuracy on train: {acc}%')
        print('Loss on test:', l_test, f'\tAccuracy on test: {acc_test}%')

### 3.3: visualising performance

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4))
ax[0].plot(np.arange(len(losses)), losses, 'b-', label='train')
ax[0].plot(np.arange(len(losses_test)), losses_test, color='orange', label='test')
ax[0].legend()
xlab, ylab = ax[0].set_xlabel('Epoch'), ax[0].set_ylabel('Loss')
ax[1].plot(np.arange(len(acces)), acces, 'b-', label='train')
ax[1].plot(np.arange(len(acces_test)), acces_test, color='orange', label='test')
ax[1].legend()
xlab, ylab = ax[1].set_xlabel('Epoch'), ax[1].set_ylabel('Accuracy')
fig.tight_layout()

---

## Implementation using the Functional API

`[from documentation]:` The Keras functional API is a way to **create models that are more flexible** than the `keras.Sequential API` (covered in the seminars). The functional API can handle **models with non-linear topology, shared layers, and even multiple inputs or outputs**.

The main idea is that **a deep learning model is usually a directed acyclic graph (DAG) of layers**. So the functional API is a way to build graphs of layers.

The basic steps are:

* specify the input layer (`tf.kerasInput`) with proper format
* specify any intermediate (hidden) layers, such as `tf.keras.layers.Dense`, using the previous layer as input to the current layer
* specify the output layer, with proper activation function
* specify the model (`tf.keras.Model`) with proper inputs and outputs

In the following example, we implement a single-layer linear regression model and use the same synthetic data for demonstration.


In [ ]:
# input layer and input shape (14 in this case, referring to our features)
input = tf.keras.Input(shape=(14,))
# the logistic regression model and proper activation function, applied to the input features
output = tf.keras.layers.Dense(1, activation='sigmoid')(input)
# the model architecture - input and output layers
model2 = tf.keras.Model(inputs=input, outputs=output)
# model parameters - optimiser, loss function, and performance metrics
model2.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.BinaryCrossentropy(), metrics=['acc'])
# we train the model for 200 few epochs
nepochs = 200
history = model2.fit(X_train, Y_train, epochs=nepochs, verbose=2, validation_data=(X_test, Y_test))

In [ ]:
nepochs = 200

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].plot(np.arange(nepochs), history.history['loss'], label='train')
ax[0].plot(np.arange(nepochs), history.history['val_loss'], color='orange', label='test')
ax[0].legend()
ax[0].title.set_text('loss')
ax[1].plot(np.arange(nepochs), history.history['acc'], label='train')
ax[1].plot(np.arange(nepochs), history.history['val_acc'], color='orange', label='test')
ax[1].title.set_text('acc')